In [1]:
require 'nn'
require 'rnn'
require 'cutorch'
require 'cunn'
require 'cunnx'

In [2]:
function build_mlp(vocab_size, embed_dim)
    local model = nn.Sequential()
    :add(nn.LookupTableMaskZero(vocab_size, embed_dim)) -- will return a sequence-length x batch-size x embedDim tensor
    :add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embedDim entries
    :add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
    :add(nn.SelectTable(-1)) -- selects last state of the LSTM
    :add(nn.Linear(embed_dim, embed_dim)) -- map last state to a score for classification
    :add(nn.ReLU())
   return model
end

function buildLSTM(vocab_size, embed_dim)
    local lstm = nn.Sequential()
    lstm:add(nn.LookupTableMaskZero(vsize, edim))
    lstm:add(nn.SplitTable(1, edim))
    lstm:add(nn.Sequencer(nn.LSTM(edim, edim)))
    lstm:add(nn.SelectTable(-1))
    return lstm
end

function buildFullModel(vocab_size, embed_dim, model, use_cuda)
    if model == 'lstm' then
        mod1 = build_mlp(vocab_size, embed_dim, usecuda)
        mod2 = build_mlp(vocab_size, embed_dim, usecuda)
        mod3 = build_mlp(vocab_size, embed_dim, usecuda)
    else         
        mod1 = buildLSTM(vocab_size, embed_dim, usecuda)
        mod2 = buildLSTM(vocab_size, embed_dim, usecuda)
        mod3 = buildLSTM(vocab_size, embed_dim, usecuda)
    end

    mod4 = nn.Sequential()
    mod4:add(nn.Linear(1, embed_dim))

    ParallelModel = nn.ParallelTable()
    ParallelModel:add(mod1)
    ParallelModel:add(mod2)
    ParallelModel:add(mod3)
    ParallelModel:add(mod4)

    FinalMLP = nn.Sequential()
    FinalMLP:add(ParallelModel)
    FinalMLP:add(nn.JoinTable(2))
    FinalMLP:add( nn.Linear(embed_dim * 4, 2) )
    FinalMLP = FinalMLP
    if use_cuda then
        return FinalMLP:cuda()
    else
        return FinalMLP
    end 

end

In [3]:
function build_data(use_cuda)    
    if use_cuda then
      Tensor = torch.CudaTensor
      LongTensor = torch.CudaLongTensor
    else
      Tensor = torch.Tensor
      LongTensor = torch.LongTensor
    end
    sentences = LongTensor{{0, 1, 3, 4}, {0, 2, 4, 3}}:t()
    summary = LongTensor{{0, 0, 1, 4}, {0, 2, 3, 1}}:t()
    query = LongTensor{{0, 0, 4, 3}, {0, 0, 0, 0}}:t()
    actions = torch.round(torch.rand(2, 1))
    yrouge = torch.rand(2, 2)
    if use_cuda then
        return sentences, summary, query, actions:cuda(), yrouge:cuda()
    else
        return sentences, summary, query, actions, yrouge
    end 
end

In [4]:
usecuda = true
model = 'lstm'

batch_size = 2
vocab_size = 4
embed_dim = 10
outputSize = 1
learning_rate = 0.1

sentences, summary, query, actions, yrouge = build_data(usecuda)
FinalMLP  = buildFullModel(vocab_size, embed_dim, model, usecuda)

criterion = nn.MSECriterion():cuda()

print(sentences)
print(summary)
print(query)
-- actions:resize(2,1)
print(actions)

print('sumval =', sentences[1]:sum())

for epoch=1, 100 do
    preds = FinalMLP:forward({sentences, summary, query, actions})
    loss = criterion:forward(preds, yrouge)
    -- This is where it fails
    grads = criterion:backward(preds, yrouge)
    FinalMLP:backward({sentences, summary, query, actions}, grads)
    FinalMLP:updateParameters(learning_rate)
    FinalMLP:zeroGradParameters()
    if (epoch % 10)==0 then 
        print(string.format("Epoch %i, loss =%6f", epoch, loss))
    end
end

print(sentences)

 0  0
 1  2
 3  4
 4  3
[torch.CudaLongTensor of size 4x2]

 0  0
 0  2
 1  3
 4  1
[torch.CudaLongTensor of size 4x2]

 0  0
 0  0
 4  0
 3  0
[torch.CudaLongTensor of size 4x2]

 1
 0
[torch.CudaTensor of size 2x1]



sumval =	0	


Epoch 10, loss =0.031530	


Epoch 20, loss =0.005607	


Epoch 30, loss =0.000926	


...

Epoch 60, loss =0.000004	


Epoch 70, loss =0.000001	


Epoch 80, loss =0.000000	


Epoch 90, loss =0.000000	


Epoch 100, loss =0.000000	
 0  0
 1  2
 3  4
 4  3
[torch.CudaLongTensor of size 4x2]



In [8]:
vsize = 4
edim = 10

In [30]:
lstm = nn.Sequential()
lstm:add(nn.LookupTableMaskZero(vsize, edim))
lstm:add(nn.SplitTable(1, edim))
lstm:add(nn.Sequencer(nn.LSTM(edim, edim)))
lstm:add(nn.SelectTable(-1))
lstm:cuda()
-- lstm:add(nn.Repeater(lstm, 10))

In [22]:
q = query:t()[1]

In [23]:
q

 0
 0
 4
 3
[torch.CudaLongTensor of size 4]



In [31]:
lstm:forward{q}

...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:67: 
In 1 module of nn.Sequential:
.../torch/install/share/lua/5.1/rnn/LookupTableMaskZero.lua:12: attempt to call field 'new' (a nil value)
stack traceback:
	.../torch/install/share/lua/5.1/rnn/LookupTableMaskZero.lua:12: in function <.../torch/install/share/lua/5.1/rnn/LookupTableMaskZero.lua:7>
	[C]: in function 'xpcall'
	...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...avierarceo/torch/install/share/lua/5.1/nn/Sequential.lua:44: in function 'f'
	[string "local f = function() return lstm:forward{q} e..."]:1: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/in